<a href="https://colab.research.google.com/github/abhiagar2019/nlp-and-transformers/blob/main/module4/NLPTransformers_Mod4Demo2_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Transformers with HuggingFace

© Data Trainers LLC. GPL v 3.0.

Author: Axel Sirota

HuggingFace is a company with a heavy open source philosophy that makes transformers readily available so you don't have to do what we did before for every application.

## Prep

In [2]:
!pip install -U datasets evaluate transformers transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [3]:
import multiprocessing
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

import sys
import keras.backend as K
import random
import os
import pandas as pd
import warnings
import time

TRACE = False
PATIENCE = 2
EPOCHS = 3
BATCH_SIZE = 256

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')

## Tokenizing and loading the dataset

In HuggingFace there are many models, and each has its own tokenizer. Lucky for us there is a class `AutoTokenizer` that does the heavylifting after we provide a checkpoint

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

raw_datasets = load_dataset("imdb")  # loads dataset raw
raw_datasets

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Notice it is a dict object with the train, test, and unsupervised datasets to play around

In [5]:
raw_datasets['train'][0]  # Let's see the first review

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

How do we know if it's positive or negative from label=0?

In [6]:
raw_datasets['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

There it is, within features we see that the index 0 is **Negative**

Now to tokenise the dataset we need to load the proper tokenizer for the model we care about. And the we are goin to apply it everywhere!

After this step the tokenizer converts the text into a Tensor of ids, each representing a diferent word in the BERT vocabulary

In [7]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    # We are using the BERT tokenizer, specifying to PAD until the end,
    # truncate if either 128 elements are met or the maximum from the model, which you get from the model card

    return tokenizer(example["text"], padding=True, truncation=True, max_length=128)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Let's see how it worked!

In [8]:
tokenized_datasets['train'][0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [9]:
tokenizer(tokenized_datasets['train'][0]['text'])

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 383

The tokenizer from BERT (well DistillBERT) converts each word into its ID according to *its* vocabulary. And notice the masking says we haven't been truncated. What we will do know is do this for all data and convert it into a TF Datasets object (which Keras accepts)

In [10]:

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=['input_ids'],
    label_cols=["label"],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=['input_ids'],
    label_cols=["label"],
    shuffle=False,
    batch_size=BATCH_SIZE,
)

In [11]:
for inputs, labels in tf_train_dataset.take(1):
  print(f' inputs: {inputs.shape}, labels: {labels.shape}')


 inputs: (256, 128), labels: (256,)


## Downloading the model and prepare for training

Now let's download the model. It is very important you use the class that starts with `TFAutoModel`. There are auto models for most tasks, so you don't have to manually add the header, for example the `TFAutoModelForSequenceClassification` adds a Dense layer (WITHOUT SOFTMAX) to do the classification

In [12]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [13]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = BATCH_SIZE
num_epochs = EPOCHS
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=1e-8, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=lr_scheduler)



In [14]:
opt

In [15]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # VERY important that HuggingFace models output logits

In [16]:
#model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model.compile(optimizer='adam', loss=loss, metrics=["accuracy"])

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=PATIENCE)


In [18]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Oh no! We have too many parameters to train! Luckily in Keras is very easy to set some layers as not trainable

In [19]:
model.layers[0].trainable = False

In [20]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 592130 (2.26 MB)
Non-trainable params: 66362880 (253.15 MB)
_________________________________________________________________


*Voilá!*

In [22]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs, callbacks=[early_stopping])

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ResourceExhaustedError: Graph execution error:

Detected at node tf_distil_bert_for_sequence_classification/distilbert/transformer/layer_._4/attention/Softmax defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages

Now we have a trained model that did transfer learning from DistillBERT

## Testing it out!

In [23]:
tokens = tokenizer(["This is the worst internet service provider", "Although most people say this is the worst, I like it"], padding=True, truncation=True, max_length=128)

In [24]:
tokens

{'input_ids': [[101, 2023, 2003, 1996, 5409, 4274, 2326, 10802, 102, 0, 0, 0, 0, 0], [101, 2348, 2087, 2111, 2360, 2023, 2003, 1996, 5409, 1010, 1045, 2066, 2009, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [25]:
model.predict(tokens['input_ids'])

1/1 [==============================] - 5s 5s/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 4.7283673, -3.8225932],
       [-3.4546597,  3.674947 ]], dtype=float32), hidden_states=None, attentions=None)

Notice the prediction where not probabilities but logits!

In [26]:
tf.math.softmax(model.predict(tokens['input_ids'])['logits'])

1/1 [==============================] - 0s 44ms/step


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9.9980670e-01, 1.9332190e-04],
       [8.0039323e-04, 9.9919957e-01]], dtype=float32)>

In [27]:
tf.math.argmax(tf.math.softmax(model.predict(tokens['input_ids'])['logits']))

1/1 [==============================] - 0s 69ms/step


<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 1])>

And the model was correct!!

In [28]:
model.evaluate(tf_validation_dataset)

98/98 [==============================] - 111s 1s/step - loss: 0.7004 - accuracy: 0.8325


[0.7003932595252991, 0.8324800133705139]